<a href="https://colab.research.google.com/github/loject/temp/blob/master/Copy_of_Untitled5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import re

from tensorflow.keras.layers import Dense, SimpleRNN, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow
print("Tensorflow version " + tensorflow.__version__)

BASE_DIR = '/content/drive/MyDrive/CNNv2'
TRAIN_FILE = '/content/drive/MyDrive/CNNv2/formated.csv'

Tensorflow version 2.8.2


In [ ]:
import glob
import os
import time, datetime

def find_in_str(st, sub_str, pos):
  last = 0
  for i in range(pos):
    last = st.find(sub_str, last + 1)
  return last

list_files = glob.glob(BASE_DIR + "/*.csv") 
open(TRAIN_FILE, 'w').close()
for i in list_files:
  print(os.path.abspath(i))
  with open(i, 'r', encoding='utf-8') as f:
      text = f.read()
      text = text.split('\n')
      for j in range(len(text)):
          elem = text[j]
          if len(elem) < 1: continue
          tm = datetime.datetime.strptime(elem.split(';')[3], '%H:%M:%S').time()
          tmp = datetime.datetime.strptime(elem.split(';')[2], '%d/%m/%y') + datetime.timedelta(hours=tm.hour, minutes=tm.minute, seconds=tm.second)
          text[j] = str(tmp.timestamp()) + elem[find_in_str(elem, ';', 4):]
      
      with open(TRAIN_FILE, 'a') as f:
          for i in text:
              f.write("%s\n" % i)

/content/drive/MyDrive/CNNv2/US1.BNGO_200120_220925.csv
/content/drive/MyDrive/CNNv2/US1.TBIO_200120_220925.csv
/content/drive/MyDrive/CNNv2/US1.TRIT_200120_220925.csv
/content/drive/MyDrive/CNNv2/US1.APP_200120_220925.csv
/content/drive/MyDrive/CNNv2/US1.GRUB_200120_220925.csv
/content/drive/MyDrive/CNNv2/US1.PFPT_200120_220925.csv
/content/drive/MyDrive/CNNv2/US1.DLO_200120_220925.csv
/content/drive/MyDrive/CNNv2/US1.MCFE_200120_220925.csv
/content/drive/MyDrive/CNNv2/US1.MIME_200120_220925.csv
/content/drive/MyDrive/CNNv2/US1.MNDY_200120_220925.csv
/content/drive/MyDrive/CNNv2/US1.TASK_200120_220925.csv
/content/drive/MyDrive/CNNv2/US1.MDRR_200120_220925.csv
/content/drive/MyDrive/CNNv2/US1.GHSI_200120_220925.csv
/content/drive/MyDrive/CNNv2/US1.ZCMD_200120_220925.csv
/content/drive/MyDrive/CNNv2/US1.SJ_200120_220925.csv
/content/drive/MyDrive/CNNv2/US1.VRME_200120_220925.csv
/content/drive/MyDrive/CNNv2/US1.KBNT_200120_220925.csv
/content/drive/MyDrive/CNNv2/US1.IMNM_200120_220925.

In [ ]:
import os
import csv
import time, datetime

def find_in_str(st, sub_str, pos):
  last = 0
  for i in range(pos):
    last = st.find(sub_str, last + 1)
  return last
  

def process_path(file_path):
  with open(file_path) as f:
    reader = csv.reader(f, delimiter=';')
    data = [tuple(i) for i in reader]
  X = np.array([data[i:i + inp_chars, :] for i in range(n)])
  Y = data[inp_chars:]  # предсказание следующего символа
  label = tensorflow.strings.split(file_path, os.sep)[-1]
  value = tensorflow.strings.split(tensorflow.io.read_file(file_path))
  return value, label

def crutch_wrap(file_path):
  return tensorflow.numpy_function(process_path, [file_path], [tensorflow.string, tensorflow.string])

dataset = tensorflow.data.Dataset.list_files(TRAIN_FILE).map(crutch_wrap)

print('asdjkf')
for (i,j) in dataset:
  try:
    print(i.numpy()[0].decode('utf-8'))
  except:
    print('error"')
    pass


asdjkf
[('1579628100.0', '1.2100000', '1.2300000', '1.2000000', '1.2000000', '5382300'), ('1579628400.0', '1.2100000', '1.2100000', '1.1900000', '1.2041000', '5995800'), ('1579628700.0', '1.2000000', '1.2097000', '1.1900000', '1.2002000', '3033100'), ('1579629000.0', '1.2000000', '1.2200000', '1.2000000', '1.2200000', '1099800'), ('1579629300.0', '1.2140000', '1.2200000', '1.2000000', '1.2196000', '2921600'), ('1579629600.0', '1.2150000', '1.2150000', '1.2050000', '1.2050000', '2460900'), ('1579629900.0', '1.2050000', '1.2100000', '1.2000000', '1.2076000', '1060800'), ('1579630200.0', '1.2012000', '1.2100000', '1.2012000', '1.2100000', '1088600'), ('1579630500.0', '1.2100000', '1.2114000', '1.1900000', '1.1950000', '5202800'), ('1579630800.0', '1.1999000', '1.2000000', '1.1850000', '1.1950000', '2491200')]


UnknownError: ignored

In [ ]:

inp_chars = 6

model = Sequential()
model.add(Input((1, 1)))  # при тренировке в рекуррентные модели keras подается сразу вся последовательность, поэтому в input теперь два числа. 1-длина последовательности, 2-размер OHE
model.add(SimpleRNN(128, activation='tanh'))  # рекуррентный слой на 500 нейронов
model.add(Dense(1, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# history = model.fit(X, Y, batch_size=32, epochs=100)

print(os.listdir(BASE_DIR))
# def buildPhrase(inp_str, str_len=50):
#     for i in range(str_len):
#         x = []
#         for j in range(i, i + inp_chars):
#             x.append(tokenizer.texts_to_matrix(inp_str[j]))  # преобразуем символы в One-Hot-encoding

#         x = np.array(x)
#         inp = x.reshape(1, inp_chars, num_characters)

#         pred = model.predict(inp)  # предсказываем OHE четвертого символа
#         d = tokenizer.index_word[pred.argmax(axis=1)[0]]  # получаем ответ в символьном представлении

#         inp_str += d  # дописываем строку

#     return inp_str


# res = buildPhrase("утренн")
# print(res)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 128)               16640     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 16,769
Trainable params: 16,769
Non-trainable params: 0
_________________________________________________________________
['war and peace.txt', 'US1.BNGO_200120_220925.txt', 'US1.TBIO_200120_220925.txt', 'US1.APP_200120_220925.txt', 'US1.PFPT_200120_220925.txt', 'US1.DLO_200120_220925.txt', 'US1.MCFE_200120_220925.txt', 'US1.MIME_200120_220925.txt', 'US1.MNDY_200120_220925.txt', 'US1.TASK_200120_220925.txt', 'US1.MDRR_200120_220925.txt', 'US1.VRME_200120_220925.txt', 'US1.KBNT_200120_220925.txt', 'US1.IMNM_200120_220925.txt', 'US1.ONCR_200120_220925.txt', 'US1.J

In [ ]:
# import tensorflow as tf
# from matplotlib import pyplot as plt
# train, test = tf.keras.datasets.mnist.load_data()
# id = 1

# print(train[0][id])
# print("test[0]")
# print(train[1][id])

# plt.figure()

# for i in range(35):
#   plt.imshow(train[0][i], cmap='gray')
#   plt.show()

# images, labels = train
# images = images/255.0
# labels = labels.astype(np.int32)

# fmnist_train_ds = tf.data.Dataset.from_tensor_slices((images, labels))
# fmnist_train_ds = fmnist_train_ds.shuffle(5000).batch(32)

# # model = tf.keras.Sequential([
# #   tf.keras.layers.Flatten(),
# #   tf.keras.layers.Dense(10)
# # ])

# # model.compile(optimizer='adam',
# #               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
# #               metrics=['accuracy'])

# # model.fit(fmnist_train_ds, epochs=2)

# # loss, accuracy = model.evaluate(fmnist_train_ds)
# # print("Loss :", loss)
# # print("Accuracy :", accuracy)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>